In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [2]:
# Paths to i2b2 dataset
i2b2Dir = '/home/jose/Downloads/i2b2/concept_assertion_relation_training_data'
trainPaths = [i2b2Dir + '/concept_assertion_relation_training_data/partners',
    i2b2Dir + '/concept_assertion_relation_training_data/beth']


In [3]:
spark = SparkSession.builder \
    .appName("assertion-status")\
    .master("local[2]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

1. i2b2 dataset needs registration prior to download.
2. training data is spread accross two directories, 'beth' and 'partners'.

In [4]:
import time


embeddingsFile = 

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

assertion = AssertionStatusApproach()\
  .setLabelCol("label") 
  .setInputCols("document") 
  .setOutputCol("assertion") 
  .setBefore(11) # set window parameters
  .setAfter(13)
  .setEmbeddingsSource(embeddingsFile, 200, WordEmbeddingsFormat.Binary)

finisher = Finisher() \
    .setInputCols(["assertion"]) \
    .setIncludeKeys(True)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    posTagger,
    finisher
  ])


NameError: name 'tokenizer' is not defined

In [5]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

+------+---------+--------------------+
|itemid|sentiment|                text|
+------+---------+--------------------+
|     1|        0|                 ...|
|     2|        0|                 ...|
|     3|        1|              omg...|
|     4|        0|          .. Omga...|
|     5|        0|         i think ...|
|     6|        0|         or i jus...|
|     7|        1|       Juuuuuuuuu...|
|     8|        0|       Sunny Agai...|
|     9|        1|      handed in m...|
|    10|        1|      hmmmm.... i...|
|    11|        0|      I must thin...|
|    12|        1|      thanks to a...|
|    13|        0|      this weeken...|
|    14|        0|     jb isnt show...|
|    15|        0|     ok thats it ...|
|    16|        0|    &lt;-------- ...|
|    17|        0|    awhhe man.......|
|    18|        1|    Feeling stran...|
|    19|        0|    HUGE roll of ...|
|    20|        0|    I just cut my...|
+------+---------+--------------------+
only showing top 20 rows



In [6]:
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")

Start fitting
Fitting is ended


In [7]:
ner_data = model.transform(data)
ner_data.show()

+------+---------+--------------------+--------------------+
|itemid|sentiment|                text|        finished_ner|
+------+---------+--------------------+--------------------+
|     1|        0|                 ...|word->is#result->...|
|     2|        0|                 ...|word->I#result->O...|
|     3|        1|              omg...|word->omg#result-...|
|     4|        0|          .. Omga...|word->..#result->...|
|     5|        0|         i think ...|word->i#result->O...|
|     6|        0|         or i jus...|word->or#result->...|
|     7|        1|       Juuuuuuuuu...|word->Juuuuuuuuuu...|
|     8|        0|       Sunny Agai...|word->Sunny#resul...|
|     9|        1|      handed in m...|word->handed#resu...|
|    10|        1|      hmmmm.... i...|word->i#result->O...|
|    11|        0|      I must thin...|word->I#result->O...|
|    12|        1|      thanks to a...|word->thanks#resu...|
|    13|        0|      this weeken...|word->this#result...|
|    14|        0|     j

In [8]:
pipeline.write().overwrite().save("./ner_pipeline")
model.write().overwrite().save("./ner_model")

In [9]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./ner_pipeline")
sameModel = PipelineModel.read().load("./ner_model")